# **Важно!** 

Домашнее задание состоит из нескольких задач, которые вам нужно решить.
*   Баллы выставляются по принципу выполнено/невыполнено.
*   За каждую выполненую задачу вы получаете баллы (количество баллов за задание указано в скобках).

**Инструкция выполнения:** Выполните задания в этом же ноутбуке (места под решения **КАЖДОЙ** задачи обозначены как **#НАЧАЛО ВАШЕГО РЕШЕНИЯ** и **#КОНЕЦ ВАШЕГО РЕШЕНИЯ**)

**Как отправить задание на проверку:** Вам необходимо сохранить ваше решение в данном блокноте и отправить итоговый **файл .IPYNB** в личном сообщении Telegram.

# **Прежде чем проверять задания:**

1. Перезапустите **ядро (restart the kernel)**: в меню, выбрать **Ядро (Kernel)**
→ **Перезапустить (Restart)**
2. Затем **Выполнить** **все ячейки (run all cells)**: в меню, выбрать **Ячейка (Cell)**
→ **Запустить все (Run All)**.

---

## Введение в Parameter Efficient Fine-Tuning (PEFT)

В данном задании основное внимание уделяется изучению и применению современных методов **Parameter Efficient Fine-Tuning (PEFT)**. Эти подходы позволяют эффективно дообучать большие языковые модели, используя лишь небольшую часть параметров, что критически важно при работе с ограниченными вычислительными ресурсами.

### Обязательные PEFT методы для изучения:

1. **LoRA (Low-Rank Adaptation)**
   - Разложение весовых матриц на произведение матриц низкого ранга
   - Ключевые гиперпараметры: `r` (rank), `alpha`, `dropout`, `target_modules`

2. **QLoRA (Quantized LoRA)** 
   - Комбинация 4-bit квантизации (NF4) с LoRA
   - Значительно снижает потребление GPU памяти

3. **AdaLoRA (Adaptive LoRA)**
   - Динамическое изменение ранга во время обучения
   - Автоматическая оптимизация распределения параметров

### Дополнительные методы (по выбору):
- **IA³ (Infused Adapter by Inhibiting and Amplifying Inner Activations)**
- **Prefix Tuning / P-Tuning v2**
- **Prompt Tuning**

### Методология сравнения PEFT подходов:

Для каждого метода необходимо измерить и сравнить:

#### Эффективность ресурсов:
- **Количество обучаемых параметров** (в % от общего числа параметров модели)
- **Потребление GPU памяти** (в GB во время обучения и инференса)
- **Время обучения** (сек/эпоху)
- **Скорость инференса** (токенов/секунду)

#### Качество результатов:
- **Основные метрики** в зависимости от задачи (ROUGE для суммаризации, BLEU для перевода)
- **Стабильность обучения** (сходимость функции потерь)
- **Качественный анализ** выходных текстов

#### Требования к отчету:
1. **Сравнительная таблица** всех протестированных методов
2. **Графики Pareto-frontier**: эффективность vs качество
3. **Обоснованные рекомендации** по выбору метода для различных сценариев
4. **Анализ компромиссов** между точностью и эффективностью

---

### Задание 1: Parameter Efficient Fine-Tuning (PEFT) моделей для суммаризации текстов

Цель задания — научиться применять современные методы параметрически-эффективного дообучения (PEFT) для настройки больших языковых моделей на задачу суммаризации текстов. Вы реализуете различные PEFT подходы: LoRA, QLoRA из библиотеки PEFT.

#### Задачи:

1. **Выбор датасета**:
   - Загрузите датасет для задачи суммаризации, например, датасет `CNN/DailyMail`, содержащий новостные статьи и их краткие содержания (референсы). Используйте библиотеку `datasets` для загрузки данных.
   - Обратите внимание на то, что вы можете использовать и другие подходящие датасеты для суммаризации (например, XSum).

2. **Предобработка данных**:
   - **Разделите** данные на обучающую и тестовую выборки.
   - **Очистите текст** от лишних символов, специальных токенов и пробелов.
   - **Подготовьте данные** в формате, подходящем для выбранной модели:
     - Для GPT-2 вам нужно будет подать текст целиком (входной текст + референсное суммирование в одном формате).
     - Для T5 модель требует форматировать входные данные в виде `summarize: <текст>` для текстов, которые нужно суммировать.

3. **Создание модели**:
   - **GPT-2**:
     - Импортируйте предобученную модель `GPT2LMHeadModel` из библиотеки Hugging Face.
     - Модель GPT-2 изначально не предобучена для задачи суммаризации, поэтому требуется её дообучение на подходящих данных.
     - Поддержите правильное управление длиной сгенерированного текста при инференсе, чтобы обеспечить краткость суммаризаций.
   
   - **T5**:
     - Для T5 используйте модель `T5ForConditionalGeneration`. T5 уже предобучена на множестве задач, включая суммаризацию, поэтому она лучше подходит для данной задачи.
     - В отличие от GPT-2, модель T5 обучена на задаче, где входной текст — это задание (например, "summarize:") + текст для обработки, а выход — краткое содержание. Это нужно учесть при подготовке данных.
     - Импортируйте модель из библиотеки Hugging Face и настройте её для использования на задаче суммаризации.

4. **Настройка параметров обучения**:
   - Настройте параметры обучения для обеих моделей:
     - Количество эпох.
     - Размер батча.
     - Скорость обучения.
     - Выберите оптимизатор (например, AdamW).
   - Для T5 используйте кросс-энтропийную функцию потерь (`CrossEntropyLoss`), так как это задача генерации текста с "условной вероятностью". Для GPT-2 используйте ту же функцию с учётом автогрегрессивного генерационного процесса.

5. **PEFT Fine-tuning модели** (основная часть задания):
   - **Обязательно** реализуйте дообучение с использованием различных PEFT методов:
     - **LoRA (Low-Rank Adaptation)**: Настройте параметры rank (r), alpha, dropout
     - **QLoRA (Quantized LoRA)**: Используйте 4-bit квантизацию с LoRA
     - **AdaLoRA**: Адаптивное изменение ранга во время обучения
     - **Дополнительно**: попробуйте IA³ (Infused Adapter by Inhibiting and Amplifying Inner Activations) или Prompt Tuning
   - Для каждого PEFT метода:
     - Настройте специфические гиперпараметры (rank, alpha, target_modules)
     - Измерьте количество обучаемых параметров
     - Зафиксируйте время обучения и потребление памяти
   - Используйте класс `Trainer` или `SFTTrainer` из библиотеки `trl` для процесса дообучения
   - Сравните результаты full fine-tuning с PEFT подходами (опционально)

6. **Инференс**:
   - Используйте обе модели для генерации кратких содержаний на тестовой выборке.
   - Подготовьте несколько примеров суммаризаций и выведите результаты для каждой модели.
   - Для инференса используйте разные стратегии декодирования:
     - **Greedy decoding** (жадный поиск).
     - **Beam search** (поиск по нескольким лучам).
     - **Sampling** (стохастическая генерация с использованием вероятностей).
   - Сравните результаты, чтобы понять, как разные стратегии влияют на качество суммаризаций.

7. **Сравнительная оценка PEFT методов**:
   - Создайте сравнительную таблицу для всех протестированных методов:
     - Количество обучаемых параметров (в % от общего числа параметров модели)
     - Время обучения на эпоху
     - Потребление GPU памяти
     - Качество суммаризации по метрикам ROUGE и BLEU
   - Оцените качество суммаризаций с использованием метрик:
     - **ROUGE-1, ROUGE-2, ROUGE-L** — для оценки точности и полноты суммаризаций
     - **BLEU** — для оценки схожести с референсным текстом
   - Проанализируйте trade-off между эффективностью обучения и качеством результата

#### Ожидаемые результаты:
- **Код с реализацией различных PEFT методов** для выбранной модели (GPT-2 или T5)
- **Сравнительный анализ** всех протестированных PEFT подходов с детальными метриками эффективности
- **Отчет** с обоснованием выбора оптимального PEFT метода для задачи суммаризации
- **Примеры суммаризаций** для каждого PEFT метода с качественным анализом различий
- **Рекомендации** по выбору PEFT подхода в зависимости от ограничений по ресурсам

#### Рекомендуемые ресурсы:
- **[PEFT Documentation](https://huggingface.co/docs/peft/index)** - основная документация библиотеки PEFT
- **[LoRA Developer Guide](https://huggingface.co/docs/peft/developer_guides/lora)** - детальное руководство по LoRA
- **[QLoRA Implementation](https://huggingface.co/docs/peft/developer_guides/quantization)** - квантизация и QLoRA
- [Документация Hugging Face Transformers](https://huggingface.co/docs/transformers/index)
- [Документация Hugging Face Datasets](https://huggingface.co/docs/datasets/index)
- **[PEFT Examples](https://github.com/huggingface/peft/tree/main/examples)** - примеры использования различных PEFT методов
- **[TRL SFTTrainer](https://huggingface.co/docs/trl/sft_trainer)** - для supervised fine-tuning

#### Критерии оценки:
- **Корректность реализации PEFT методов** (30%) - правильная настройка и применение различных PEFT подходов
- **Полнота сравнительного анализа** (25%) - детальное сравнение методов по всем указанным метрикам
- **Качество сгенерированных суммаризаций** (20%) - оценка по ROUGE/BLEU метрикам
- **Глубина анализа и выводов** (15%) - обоснованные рекомендации по выбору PEFT метода
- **Четкость и структурированность отчета** (10%) - качество документации процесса и результатов

In [ ]:
import torch

print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("CUDA version:", torch.version.cuda)
    print("cuDNN version:", torch.backends.cudnn.version())
    print("Device count:", torch.cuda.device_count())
    print("Current device index:", torch.cuda.current_device())
    print("Device name:", torch.cuda.get_device_name(0))
    print("Memory allocated:", torch.cuda.memory_allocated() // 1024**2, "MB")
    print("Memory reserved:", torch.cuda.memory_reserved() // 1024**2, "MB")

x = torch.rand(3, 3)
print("Default tensor device:", x.device)

if torch.cuda.is_available():
    x_gpu = torch.rand(3, 3).cuda()
    print("Tensor on GPU device:", x_gpu.device)


Torch version: 2.7.1+cu118
CUDA available: True
CUDA version: 11.8
cuDNN version: 90100
Device count: 1
Current device index: 0
Device name: NVIDIA GeForce RTX 4070
Memory allocated: 11699 MB
Memory reserved: 14218 MB
Default tensor device: cpu
Tensor on GPU device: cuda:0


In [ ]:
import os
import time
from typing import Optional, Dict, Any, List

import torch
from transformers import (
    AutoTokenizer, T5ForConditionalGeneration, GPT2LMHeadModel,
    Trainer, TrainingArguments, DataCollatorForSeq2Seq
)
from datasets import load_dataset
import evaluate
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, AdaLoraConfig
from transformers import BitsAndBytesConfig

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)
if DEVICE.type == "cuda":
    print("CUDA device name:", torch.cuda.get_device_name(torch.cuda.current_device()))

def count_trainable_parameters(model: torch.nn.Module) -> Dict[str, Any]:
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return {"total": total, "trainable": trainable, "trainable_pct": 100.0 * trainable / total}

def gpu_memory_used() -> Dict[str, int]:
    if not torch.cuda.is_available():
        return {"available": 0}
    idx = torch.cuda.current_device()
    return {
        "allocated_bytes": torch.cuda.memory_allocated(idx),
        "max_allocated_bytes": torch.cuda.max_memory_allocated(idx),
        "reserved_bytes": torch.cuda.memory_reserved(idx),
        "max_reserved_bytes": torch.cuda.max_memory_reserved(idx),
    }

def load_dataset_small(dataset_name="cnn_dailymail", dataset_config="3.0.0", sample_small: Optional[int]=None):
    ds = load_dataset(dataset_name, dataset_config)
    train = ds["train"]
    valid = ds["validation"] if "validation" in ds else ds["test"]
    if sample_small:
        train = train.select(range(min(sample_small, len(train))))
        valid = valid.select(range(min(sample_small // 10, len(valid))))
    return train, valid

def clean_text(example):
    for key in ["article", "highlights", "summary", "document", "text"]:
        if key in example and isinstance(example[key], str):
            example[key] = example[key].strip()
    return example

def preprocess_t5(tokenizer, examples, max_input=512, max_target=128):
    inputs = ["summarize: " + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=max_input, truncation=True, padding="max_length")
    labels = tokenizer(text_target=examples["highlights"], max_length=max_target, truncation=True, padding="max_length")["input_ids"]
    labels = [[-100 if tok == tokenizer.pad_token_id else tok for tok in seq] for seq in labels]
    model_inputs["labels"] = labels
    return model_inputs

def preprocess_gpt2(tokenizer, examples, max_length=1024):
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token or "<|pad|>"
    inputs = []
    for article, summary in zip(examples["article"], examples["highlights"]):
        text = article.strip() + tokenizer.eos_token + summary.strip() + tokenizer.eos_token
        inputs.append(text)
    enc = tokenizer(inputs, max_length=max_length, truncation=True, padding="max_length")
    enc["labels"] = [[-100 if tok == tokenizer.pad_token_id else tok for tok in seq] for seq in enc["input_ids"]]
    return enc

def create_t5(model_name="t5-base", peft_type="lora", use_4bit=False, target_modules=None, total_steps=None):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token or "<pad>"
    if use_4bit:
        bnb = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.float16)
        model = T5ForConditionalGeneration.from_pretrained(model_name, quantization_config=bnb)
        model = prepare_model_for_kbit_training(model)
    else:
        model = T5ForConditionalGeneration.from_pretrained(model_name)
    if peft_type == "lora":
        config = LoraConfig(r=8, lora_alpha=32, target_modules=target_modules or ["q","v"], lora_dropout=0.1, bias="none", task_type="SEQ_2_SEQ_LM")
        model = get_peft_model(model, config)
    elif peft_type == "adalora":
        total_step = total_steps if total_steps and total_steps > 0 else 1
        config = AdaLoraConfig(
            r=16,
            target_modules=target_modules or ["q","v"],
            lora_alpha=32,
            init_r=4,
            tinit=100,
            tfinal=1000,
            total_step=total_step
        )
        model = get_peft_model(model, config)
    return tokenizer, model

def create_gpt2(model_name="gpt2", peft_type="lora", use_4bit=False, target_modules=None, total_steps=None):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({"pad_token": "<|pad|>"})
    if use_4bit:
        bnb = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.float16)
        model = GPT2LMHeadModel.from_pretrained(model_name, quantization_config=bnb)
        model = prepare_model_for_kbit_training(model)
    else:
        model = GPT2LMHeadModel.from_pretrained(model_name)
    model.resize_token_embeddings(len(tokenizer))
    if peft_type == "lora":
        config = LoraConfig(r=8, lora_alpha=32, target_modules=target_modules or ["c_attn"], lora_dropout=0.1, bias="none", task_type="CAUSAL_LM")
        model = get_peft_model(model, config)
    elif peft_type == "adalora":
        total_step = total_steps if total_steps and total_steps > 0 else 1
        config = AdaLoraConfig(
            r=16,
            target_modules=target_modules or ["c_attn"],
            lora_alpha=32,
            init_r=4,
            tinit=100,
            tfinal=1000,
            total_step=total_step
        )
        model = get_peft_model(model, config)
    return tokenizer, model

def make_args(output_dir, train_batch=8, eval_batch=8, epochs=3, lr=5e-5):
    return TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=train_batch,
        per_device_eval_batch_size=eval_batch,
        num_train_epochs=epochs,
        learning_rate=lr,
        weight_decay=0.01,
        fp16=False,
        logging_steps=50,
        eval_strategy="epoch",
        save_strategy="epoch"
    )

def build_trainer(model, tokenizer, train_ds, eval_ds, is_seq2seq=True, output_dir="./out", train_batch=8, eval_batch=8, epochs=3):
    args = make_args(output_dir, train_batch, eval_batch, epochs)
    if is_seq2seq:
        collator = DataCollatorForSeq2Seq(tokenizer, model=model, label_pad_token_id=-100, padding=True)
    else:
        collator = None
    return Trainer(model=model, args=args, train_dataset=train_ds, eval_dataset=eval_ds, tokenizer=tokenizer, data_collator=collator)

def compute_metrics(preds: List[str], refs: List[str]) -> Dict[str, Any]:
    rouge = evaluate.load("rouge")
    bleu = evaluate.load("bleu")
    rouge_res = rouge.compute(predictions=preds, references=refs)
    bleu_res = bleu.compute(predictions=preds, references=refs)
    return {"rouge1": rouge_res["rouge1"], "rouge2": rouge_res["rouge2"], "rougeL": rouge_res["rougeL"], "bleu": bleu_res["bleu"]}

def run_peft(model_name, model_type="t5", peft_type="lora", sample_small=2000, output_dir="./out"):
    train_ds, valid_ds = load_dataset_small(sample_small=sample_small)
    train_ds = train_ds.filter(lambda x: x.get("highlights") is not None and x.get("highlights").strip() != "")
    valid_ds = valid_ds.filter(lambda x: x.get("highlights") is not None and x.get("highlights").strip() != "")
    train_ds = train_ds.map(clean_text)
    valid_ds = valid_ds.map(clean_text)

    batch_size = 4
    num_epochs = 3
    total_steps = max(1, (len(train_ds) // batch_size) * num_epochs)

    if model_type == "t5":
        tokenizer, model = create_t5(model_name, peft_type, total_steps=total_steps)
        preprocess = preprocess_t5
        is_seq2seq = True
    else:
        tokenizer, model = create_gpt2(model_name, peft_type, total_steps=total_steps)
        preprocess = preprocess_gpt2
        is_seq2seq = False

    tokenized_train = train_ds.map(lambda ex: preprocess(tokenizer, ex), batched=True, remove_columns=train_ds.column_names)
    tokenized_valid = valid_ds.map(lambda ex: preprocess(tokenizer, ex), batched=True, remove_columns=valid_ds.column_names)

    trainer = build_trainer(model, tokenizer, tokenized_train, tokenized_valid, is_seq2seq=is_seq2seq, output_dir=output_dir, train_batch=batch_size, eval_batch=batch_size, epochs=num_epochs)

    collator = DataCollatorForSeq2Seq(tokenizer, model=model, label_pad_token_id=-100, padding=True) if is_seq2seq else None
    if collator is not None:
        sample = tokenized_train.select(range(min(4, len(tokenized_train))))
        batch = collator(sample)
        print("Sanity batch keys:", list(batch.keys()))
        print({k: getattr(v, "shape", None) for k, v in batch.items() if hasattr(v, "shape")})
        labels = batch.get("labels")
        if labels is not None:
            all_neg100 = (labels == -100).all(dim=1)
            print("Any sequence fully -100 (bad)?:", all_neg100.any().item())

    t0 = time.time()
    trainer.train()
    train_time = time.time() - t0

    model.to(DEVICE)
    model.eval()

    samples = valid_ds.select(range(min(5, len(valid_ds))))
    inputs = ["summarize: " + s["article"] if model_type == "t5" else s["article"] for s in samples]
    enc = tokenizer(inputs, return_tensors="pt", truncation=True, padding="max_length", max_length=512)
    enc = {k: v.to(DEVICE) for k, v in enc.items()}

    outputs_beam = model.generate(**enc, max_new_tokens=128, num_beams=4)
    outputs_greedy = model.generate(**enc, max_new_tokens=128)
    outputs_sample = model.generate(**enc, max_new_tokens=128, do_sample=True, top_k=50, top_p=0.95)

    dec_beam = tokenizer.batch_decode(outputs_beam, skip_special_tokens=True)
    dec_greedy = tokenizer.batch_decode(outputs_greedy, skip_special_tokens=True)
    dec_sample = tokenizer.batch_decode(outputs_sample, skip_special_tokens=True)
    refs = [s["highlights"] for s in samples]

    metrics_beam = compute_metrics(dec_beam, refs)
    metrics_greedy = compute_metrics(dec_greedy, refs)
    metrics_sample = compute_metrics(dec_sample, refs)

    param_summary = count_trainable_parameters(model)
    gpu_mem = gpu_memory_used()

    return {
        "param_summary": param_summary,
        "train_time_s": train_time,
        "gpu_mem": gpu_mem,
        "metrics_beam": metrics_beam,
        "metrics_greedy": metrics_greedy,
        "metrics_sample": metrics_sample,
        "examples": {"inputs": inputs, "beam": dec_beam, "greedy": dec_greedy, "sample": dec_sample}
    }

def compare_peft_methods():
    results = {}
    for model_type in ["t5", "gpt2"]:
        for peft_type in ["lora", "adalora"]:
            key = f"{model_type}_{peft_type}"
            out_dir = f"./out/{key}"
            results[key] = run_peft("t5-large" if model_type == "t5" else "gpt2", model_type=model_type, peft_type=peft_type, output_dir=out_dir, sample_small=2000)
    return results


d:\NLP\5\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda
CUDA device name: NVIDIA GeForce RTX 4070


In [2]:
results_t5_lora = run_peft(
    model_name="t5-large",
    model_type="t5",
    peft_type="lora",
    output_dir="./out/t5_lora",
    sample_small=2000
)


C:\Users\igor\AppData\Local\Temp\ipykernel_15380\3481498702.py:146: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  return Trainer(model=model, args=args, train_dataset=train_ds, eval_dataset=eval_ds, tokenizer=tokenizer, data_collator=collator)


Sanity batch keys: ['input_ids', 'attention_mask', 'labels', 'decoder_input_ids']
{'input_ids': torch.Size([4, 512]), 'attention_mask': torch.Size([4, 512]), 'labels': torch.Size([4, 128]), 'decoder_input_ids': torch.Size([4, 128])}
Any sequence fully -100 (bad)?: False


Epoch,Training Loss,Validation Loss
1,1.348400,1.646238
2,1.295700,1.641138
3,1.292400,1.641412


In [3]:
results_t5_adalora = run_peft(
    model_name="t5-large",
    model_type="t5",
    peft_type="adalora",
    output_dir="./out/t5_adalora",
    sample_small=2000
)


d:\NLP\5\venv\Lib\site-packages\peft\tuners\adalora\config.py:96: UserWarning: Note that `r` is not used in AdaLora and will be ignored.If you intended to set the initial rank, use `init_r` instead.
  warnings.warn(
C:\Users\igor\AppData\Local\Temp\ipykernel_15380\3481498702.py:146: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  return Trainer(model=model, args=args, train_dataset=train_ds, eval_dataset=eval_ds, tokenizer=tokenizer, data_collator=collator)


Sanity batch keys: ['input_ids', 'attention_mask', 'labels', 'decoder_input_ids']
{'input_ids': torch.Size([4, 512]), 'attention_mask': torch.Size([4, 512]), 'labels': torch.Size([4, 128]), 'decoder_input_ids': torch.Size([4, 128])}
Any sequence fully -100 (bad)?: False


Epoch,Training Loss,Validation Loss
1,1.545700,1.829451
2,1.373000,1.699204
3,1.382900,1.684347


In [4]:
results_gpt2_lora = run_peft(
    model_name="gpt2",
    model_type="gpt2",
    peft_type="lora",
    output_dir="./out/gpt2_lora",
    sample_small=2000
)


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
d:\NLP\5\venv\Lib\site-packages\peft\tuners\lora\layer.py:2174: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
Map: 100%|██████████| 200/200 [00:00<00:00, 467.25 examples/s]
C:\Users\igor\AppData\Local\Temp\ipykernel_15380\3481498702.py:146: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  return Trainer(model=model, args=args, train_dataset=train_ds, eval_dataset=eval_ds, tokenizer=tokenizer, data_collator=collator)
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordi

Epoch,Training Loss,Validation Loss
1,3.028700,3.012906
2,2.971200,3.003466
3,2.949600,3.001813


d:\NLP\5\venv\Lib\site-packages\peft\utils\save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
d:\NLP\5\venv\Lib\site-packages\peft\utils\save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
d:\NLP\5\venv\Lib\site-packages\peft\utils\save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was d

In [5]:
results_gpt2_lora = run_peft(
    model_name="gpt2",
    model_type="gpt2",
    peft_type="lora",
    output_dir="./out/gpt2_lora",
    sample_small=2000
)


d:\NLP\5\venv\Lib\site-packages\peft\tuners\lora\layer.py:2174: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
C:\Users\igor\AppData\Local\Temp\ipykernel_15380\3481498702.py:146: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  return Trainer(model=model, args=args, train_dataset=train_ds, eval_dataset=eval_ds, tokenizer=tokenizer, data_collator=collator)
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 50257}.


Epoch,Training Loss,Validation Loss
1,3.028700,3.012906
2,2.971200,3.003466
3,2.949600,3.001813


d:\NLP\5\venv\Lib\site-packages\peft\utils\save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
d:\NLP\5\venv\Lib\site-packages\peft\utils\save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
d:\NLP\5\venv\Lib\site-packages\peft\utils\save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was d

In [6]:
results_gpt2_adalora = run_peft(
    model_name="gpt2",
    model_type="gpt2",
    peft_type="adalora",
    output_dir="./out/gpt2_adalora",
    sample_small=2000
)


d:\NLP\5\venv\Lib\site-packages\peft\tuners\adalora\config.py:96: UserWarning: Note that `r` is not used in AdaLora and will be ignored.If you intended to set the initial rank, use `init_r` instead.
  warnings.warn(
d:\NLP\5\venv\Lib\site-packages\peft\tuners\adalora\model.py:210: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
C:\Users\igor\AppData\Local\Temp\ipykernel_15380\3481498702.py:146: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  return Trainer(model=model, args=args, train_dataset=train_ds, eval_dataset=eval_ds, tokenizer=tokenizer, data_collator=collator)
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 50257}.


Epoch,Training Loss,Validation Loss
1,3.179100,3.096125
2,3.123900,3.078792
3,3.080000,3.072660


d:\NLP\5\venv\Lib\site-packages\peft\utils\save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
d:\NLP\5\venv\Lib\site-packages\peft\utils\save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
d:\NLP\5\venv\Lib\site-packages\peft\utils\save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was d

In [9]:
results = {
    "t5_lora": results_t5_lora,
    "t5_adalora": results_t5_adalora,
    "gpt2_lora": results_gpt2_lora,
    "gpt2_adalora": results_gpt2_adalora,
}

for k, v in results.items():
    print(f"\nResults for {k}:")
    print("Trainable params:", v["param_summary"])
    print("Train time (s):", v["train_time_s"])
    print("GPU memory:", v["gpu_mem"])
    print("Beam metrics:", v["metrics_beam"])
    print("Greedy metrics:", v["metrics_greedy"])
    print("Sample metrics:", v["metrics_sample"])
    print("Examples:")
    for i in range(len(v["examples"]["inputs"])):
        print("Input:", v["examples"]["inputs"][i][:200])
        print("Beam:", v["examples"]["beam"][i])
        print("Greedy:", v["examples"]["greedy"][i])
        print("Sample:", v["examples"]["sample"][i])



Results for t5_lora:
Trainable params: {'total': 740027392, 'trainable': 2359296, 'trainable_pct': 0.31881198257050464}
Train time (s): 1121.1335113048553
GPU memory: {'allocated_bytes': 2996615680, 'max_allocated_bytes': 12045687808, 'reserved_bytes': 12364808192, 'max_reserved_bytes': 12364808192}
Beam metrics: {'rouge1': np.float64(0.3122651346759172), 'rouge2': np.float64(0.10260241847467011), 'rougeL': np.float64(0.23372992891064373), 'bleu': 0.08166357214354078}
Greedy metrics: {'rouge1': np.float64(0.32469812685812094), 'rouge2': np.float64(0.10130718954248366), 'rougeL': np.float64(0.2388686367811513), 'bleu': 0.09410711373430254}
Sample metrics: {'rouge1': np.float64(0.2180936742513097), 'rouge2': np.float64(0.055714285714285716), 'rougeL': np.float64(0.18850468086921288), 'bleu': 0.0}
Examples:
Input: summarize: (CNN)Share, and your gift will be multiplied. That may sound like an esoteric adage, but when Zully Broussard selflessly decided to give one of her kidneys to a stra

### Задание 2: Применение различных PEFT методов для задачи машинного перевода

**Цель задания**: Сравнить эффективность различных Parameter Efficient Fine-Tuning (PEFT) подходов для задачи автоматического перевода с английского на русский. В рамках задания необходимо реализовать и сравнить минимум 3 различных PEFT метода, проанализировать их влияние на качество перевода и эффективность обучения.

#### Задачи:

1. **Выбор датасета**:
   - Загрузите параллельный датасет для перевода, например, [Opus Books](https://huggingface.co/datasets/Helsinki-NLP/opus_books/viewer/en-ru), который содержит тексты на английском языке и их переводы на русский.
   - Используйте библиотеку `datasets` для загрузки и обработки данных. Убедитесь, что данные содержат параллельные тексты для обучения модели переводу.

2. **Предобработка данных**:
   - **Разделите данные** на обучающую и тестовую выборки (например, 80% для обучения и 20% для тестирования).
   - **Очистите текст**, удалив лишние пробелы и специальные символы, которые могут повлиять на обучение модели.
   - **Подготовьте данные** в формате, подходящем для выбранной модели:
     - Для GPT-2 данные должны быть в виде последовательности токенов, где исходный текст и перевод разделены специальными символами (например, `<|startoftext|>` для начала текста и `<|endoftext|>` для его конца).
     - Для T5 данные подаются в формате задачи перевода: входной текст начинается с задания `"translate English to Russian: <текст на английском>"`, а на выходе модель должна сгенерировать перевод.

3. **Создание модели**:
   - **GPT-2**:
     - Импортируйте предобученную модель `GPT2LMHeadModel` из библиотеки Hugging Face.
     - GPT-2 изначально не обучена для задачи перевода, поэтому нужно будет использовать специальную подготовку данных и дообучение на параллельных текстах.
     - Убедитесь, что модель настроена для генерации текста, ограничивая длину вывода для перевода.
   
   - **T5**:
     - Импортируйте модель `T5ForConditionalGeneration`, которая предобучена на множестве задач, включая перевод. T5 — это модель с условной генерацией, которая использует специальную задачу (`task`) для перевода.
     - Подготовьте модель для выполнения задачи перевода с английского на русский, используя предобученные веса и формат входных данных.

4. **Настройка параметров обучения**:
   - Настройте параметры обучения для обеих моделей:
     - Количество эпох (например, 3-5 эпох).
     - Размер батча (например, 16 или 32).
     - Скорость обучения (рекомендуется начать с 5e-5 и адаптировать в зависимости от потерь на валидации).
   - Используйте подходящие оптимизаторы, такие как AdamW, и функцию потерь для задачи перевода:
     - Для T5 подойдёт стандартная кросс-энтропийная функция потерь.
     - Для GPT-2 используйте ту же функцию с учётом последовательной генерации текста (автогрегрессии).

5. **Сравнительное исследование PEFT методов** (ключевая часть задания):
   - **Обязательно** реализуйте и сравните следующие PEFT подходы:
     - **LoRA**: Настройте различные значения rank (4, 8, 16, 32), alpha (16, 32, 64)
     - **QLoRA**: Комбинация 4-bit квантизации с LoRA для экономии памяти
     - **AdaLoRA**: Адаптивное изменение ранга с бюджетом параметров
     - **Дополнительные методы** (на выбор): IA³, Prefix Tuning, P-Tuning v2, или (IA)³
   - Для каждого PEFT метода проведите **grid search** по ключевым гиперпараметрам
   - **Baseline**: обязательно сравните с полным fine-tuning (если позволяют ресурсы)
   - Зафиксируйте для каждого эксперимента:
     - Процент обучаемых параметров от общего числа
     - Потребление GPU памяти (в GB)
     - Время обучения на эпоху
     - Скорость инференса (токенов/сек)
   - Используйте `SFTTrainer` или `Trainer` для стабильного процесса обучения

6. **Инференс**:
   - Используйте дообученную модель для перевода текстов с английского на русский на тестовой выборке.
   - Подготовьте несколько примеров для перевода и выведите результаты:
     - Для GPT-2 используйте автогрегрессивное декодирование текста.
     - Для T5 применяйте стандартные стратегии декодирования (например, greedy decoding или beam search).
   
7. **Комплексная оценка и анализ PEFT методов**:
   - **Автоматические метрики**:
     - **BLEU** (corpus-level и sentence-level) — основная метрика для машинного перевода
     - **chrF** — character-level F-score для более точной оценки морфологически богатых языков
     - **COMET** — нейронная метрика качества перевода (если позволяют ресурсы)
     - **ROUGE** — для дополнительной оценки похожести
   - **Создайте детальную сравнительную таблицу**:
     - Эффективность (% параметров, время, память) vs Качество (BLEU, chrF)
     - Pareto-frontier анализ: какие методы обеспечивают лучший trade-off
   - **Качественный анализ**: 
     - Проанализируйте примеры переводов от каждого PEFT метода
     - Определите типы ошибок, характерные для каждого подхода
     - Оцените стабильность качества на различных типах текстов

#### Ожидаемые результаты:
- **Исследовательский код** с реализацией всех протестированных PEFT методов
- **Научно-обоснованный отчет** с детальным сравнением эффективности PEFT подходов
- **Визуализации**: графики Pareto-frontier, сравнительные диаграммы по метрикам
- **Практические рекомендации**: когда использовать каждый PEFT метод в зависимости от ограничений
- **Воспроизводимые результаты**: четкие инструкции по повторению экспериментов

#### Рекомендуемые ресурсы:
- **[PEFT Documentation](https://huggingface.co/docs/peft/index)** - полная документация библиотеки PEFT
- **[LoRA Paper & Implementation](https://arxiv.org/abs/2106.09685)** - оригинальная статья LoRA
- **[QLoRA Paper](https://arxiv.org/abs/2305.14314)** - квантизованный LoRA подход
- **[AdaLoRA Paper](https://arxiv.org/abs/2303.10512)** - адаптивный LoRA
- **[PEFT Task Guides](https://huggingface.co/docs/peft/task_guides/translation)** - гайды по применению PEFT для разных задач
- [Документация Hugging Face Transformers](https://huggingface.co/docs/transformers/index)
- [Документация Hugging Face Datasets](https://huggingface.co/docs/datasets/index)
- **[BitsAndBytes](https://huggingface.co/docs/bitsandbytes/index)** - для квантизации в QLoRA
- **[TRL Library](https://huggingface.co/docs/trl/index)** - для эффективного обучения

#### Критерии оценки:
- **Корректность реализации PEFT методов** (35%) - правильная настройка минимум 3 различных PEFT подходов
- **Качество сравнительного анализа** (25%) - детальное сравнение по всем метрикам эффективности и качества
- **Научная обоснованность выводов** (20%) - аргументированные рекомендации по выбору методов
- **Воспроизводимость результатов** (10%) - четкие инструкции и фиксированные семена
- **Качество перевода** (10%) - достижение конкурентоспособных результатов по BLEU/chrF метрикам

1

In [ ]:
from datasets import load_dataset
dataset = load_dataset("Helsinki-NLP/opus_books", "en-ru")
print(dataset)
print(dataset['train'][0])


d:\NLP\5\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 17496
    })
})
{'id': '0', 'translation': {'en': 'Anna Karenina', 'ru': 'Анна Каренина'}}


In [ ]:
from datasets import DatasetDict, Dataset
import re
from sklearn.model_selection import train_test_split

def clean_text(text):
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)
    return text

data = [{'en': clean_text(x['translation']['en']),
         'ru': clean_text(x['translation']['ru'])} for x in dataset['train']]

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
train_dataset = Dataset.from_list(train_data)
test_dataset = Dataset.from_list(test_data)
dataset_dict = DatasetDict({'train': train_dataset, 'test': test_dataset})

print(dataset_dict)
print(dataset_dict['train'][0])


DatasetDict({
    train: Dataset({
        features: ['en', 'ru'],
        num_rows: 13996
    })
    test: Dataset({
        features: ['en', 'ru'],
        num_rows: 3500
    })
})
{'en': 'The rye – after he had so long held out for a certain price – was sold fifty kopeks a chetvert cheaper than had been offered him a month ago.', 'ru': 'Рожь, цену на которую он так долго выдерживал, была продана пятьюдесятью копейками на четверть дешевле, чем за нее давали месяц тому назад.'}


In [ ]:
def prepare_gpt2_example(example):
    return {
        "text": f"<|startoftext|> {example['en']} <|sep|> {example['ru']} <|endoftext|>"
    }

gpt2_train = train_dataset.map(prepare_gpt2_example)
gpt2_test = test_dataset.map(prepare_gpt2_example)

print(gpt2_train[0])


Map: 100%|██████████| 3500/3500 [00:00<00:00, 44762.02 examples/s]

{'en': 'The rye – after he had so long held out for a certain price – was sold fifty kopeks a chetvert cheaper than had been offered him a month ago.', 'ru': 'Рожь, цену на которую он так долго выдерживал, была продана пятьюдесятью копейками на четверть дешевле, чем за нее давали месяц тому назад.', 'text': '<|startoftext|> The rye – after he had so long held out for a certain price – was sold fifty kopeks a chetvert cheaper than had been offered him a month ago. <|sep|> Рожь, цену на которую он так долго выдерживал, была продана пятьюдесятью копейками на четверть дешевле, чем за нее давали месяц тому назад. <|endoftext|>'}


In [4]:
def prepare_t5_example(example):
    return {
        "input_text": f"translate English to Russian: {example['en']}",
        "target_text": example['ru']
    }

t5_train = train_dataset.map(prepare_t5_example)
t5_test = test_dataset.map(prepare_t5_example)

print(t5_train[0])


Map: 100%|██████████| 3500/3500 [00:00<00:00, 43488.11 examples/s]

{'en': 'The rye – after he had so long held out for a certain price – was sold fifty kopeks a chetvert cheaper than had been offered him a month ago.', 'ru': 'Рожь, цену на которую он так долго выдерживал, была продана пятьюдесятью копейками на четверть дешевле, чем за нее давали месяц тому назад.', 'input_text': 'translate English to Russian: The rye – after he had so long held out for a certain price – was sold fifty kopeks a chetvert cheaper than had been offered him a month ago.', 'target_text': 'Рожь, цену на которую он так долго выдерживал, была продана пятьюдесятью копейками на четверть дешевле, чем за нее давали месяц тому назад.'}


In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

gpt2_model_name = "gpt2"
gpt2_tokenizer = GPT2Tokenizer.from_pretrained(gpt2_model_name)
gpt2_model = GPT2LMHeadModel.from_pretrained(gpt2_model_name)

special_tokens = {"pad_token": "<|pad|>", "bos_token": "<|startoftext|>", "eos_token": "<|endoftext|>", "additional_special_tokens": ["<|sep|>"]}
num_added = gpt2_tokenizer.add_special_tokens(special_tokens)
gpt2_model.resize_token_embeddings(len(gpt2_tokenizer))

print(f"Added {num_added} special tokens")


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Added 3 special tokens


In [6]:
from transformers import T5TokenizerFast

t5_tokenizer = T5TokenizerFast.from_pretrained("t5-small")


In [ ]:
from transformers import DataCollatorForLanguageModeling

def tokenize_gpt2(example):
    return gpt2_tokenizer(example['text'], truncation=True, max_length=512)

gpt2_train_tok = gpt2_train.map(tokenize_gpt2, batched=True)
gpt2_test_tok = gpt2_test.map(tokenize_gpt2, batched=True)

gpt2_data_collator = DataCollatorForLanguageModeling(
    tokenizer=gpt2_tokenizer, 
    mlm=False
)


Map: 100%|██████████| 3500/3500 [00:01<00:00, 2285.62 examples/s]


In [ ]:
def tokenize_t5(example):
    inputs = t5_tokenizer(example['input_text'], truncation=True, padding="max_length", max_length=512)
    targets = t5_tokenizer(example['target_text'], truncation=True, padding="max_length", max_length=512)
    inputs["labels"] = targets["input_ids"]
    return inputs

t5_train_tok = t5_train.map(tokenize_t5, batched=True)
t5_test_tok = t5_test.map(tokenize_t5, batched=True)


Map: 100%|██████████| 3500/3500 [00:00<00:00, 6018.48 examples/s]


In [ ]:
from peft import LoraConfig, get_peft_model
gpt2_lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["c_attn"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

gpt2_model_lora = get_peft_model(gpt2_model, gpt2_lora_config)


d:\NLP\5\venv\Lib\site-packages\peft\tuners\lora\layer.py:2174: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [11]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

t5_model_name = "t5-small"
t5_tokenizer = T5Tokenizer.from_pretrained(t5_model_name)
t5_model = T5ForConditionalGeneration.from_pretrained(t5_model_name)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
t5_lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)

t5_model_lora = get_peft_model(t5_model, t5_lora_config)


In [ ]:
from transformers import Trainer, TrainingArguments

training_args_gpt2 = TrainingArguments(
    output_dir="./gpt2_lora",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=10,
    learning_rate=5e-5,
    logging_steps=50,
    save_total_limit=1,
    fp16=False,
    gradient_accumulation_steps=8,
)

trainer_gpt2 = Trainer(
    model=gpt2_model_lora,
    args=training_args_gpt2,
    train_dataset=gpt2_train_tok,
    eval_dataset=gpt2_test_tok,
    tokenizer=gpt2_tokenizer,
    data_collator=gpt2_data_collator,
)


C:\Users\igor\AppData\Local\Temp\ipykernel_20144\3435328781.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_gpt2 = Trainer(


In [30]:
trainer_gpt2.train()


Step,Training Loss
50,2.268600
100,2.260500
150,2.274800
200,2.261400
250,2.264800
300,2.268000
350,2.251900
400,2.278000
450,2.267000
500,2.258400


d:\NLP\5\venv\Lib\site-packages\peft\utils\save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
d:\NLP\5\venv\Lib\site-packages\peft\utils\save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
d:\NLP\5\venv\Lib\site-packages\peft\utils\save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
d:\NLP\5\venv\Lib\site-packages\peft\utils\save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
d:\NLP\5\venv\Lib\site-packages\peft\utils\save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
d:\NLP\5\venv\Lib\si

TrainOutput(global_step=8750, training_loss=2.224503541782924, metrics={'train_runtime': 2090.5006, 'train_samples_per_second': 66.95, 'train_steps_per_second': 4.186, 'total_flos': 1.3302148574232576e+16, 'train_loss': 2.224503541782924, 'epoch': 10.0})

In [ ]:
training_args_t5 = TrainingArguments(
    output_dir="./t5_lora",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=15,
    learning_rate=5e-5,
    logging_steps=50,
    save_total_limit=1,
    fp16=False,
)

trainer_t5 = Trainer(
    model=t5_model_lora,
    args=training_args_t5,
    train_dataset=t5_train_tok,
    eval_dataset=t5_test_tok,
    tokenizer=t5_tokenizer,
)

C:\Users\igor\AppData\Local\Temp\ipykernel_20144\3568226505.py:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_t5 = Trainer(


In [34]:
trainer_t5.train()


Step,Training Loss
50,0.352800
100,0.342900
150,0.338300
200,0.305800
250,0.381200
300,0.343100
350,0.375200
400,0.319800
450,0.322500
500,0.318200


TrainOutput(global_step=104970, training_loss=0.32576767657159045, metrics={'train_runtime': 5610.3912, 'train_samples_per_second': 37.42, 'train_steps_per_second': 18.71, 'total_flos': 2.860385707229184e+16, 'train_loss': 0.32576767657159045, 'epoch': 15.0})

In [ ]:
from torch import cuda

gpt2_model_lora.eval()
device = "cuda" if cuda.is_available() else "cpu"
gpt2_model_lora.to(device)

def gpt2_translate(text):
    input_text = f"<|startoftext|> {text} <|sep|>"
    input_ids = gpt2_tokenizer.encode(input_text, return_tensors="pt").to(device)
    output_ids = gpt2_model_lora.generate(
        input_ids,
        max_length=200,
        num_beams=5,
        early_stopping=True,
        pad_token_id=gpt2_tokenizer.pad_token_id
    )
    output_text = gpt2_tokenizer.decode(output_ids[0], skip_special_tokens=True)

    if "<|sep|>" in output_text:
        output_text = output_text.split("<|sep|>")[1].strip()
    return output_text

for i in range(5):
    ex = gpt2_test[i]
    text_en = ex['text'].split("<|sep|>")[0].replace("<|startoftext|>", "").strip()
    text_gt = ex['text'].split("<|sep|>")[1].replace("<|endoftext|>", "").strip()
    print("EN:", text_en)
    print("GT:", text_gt)
    print("PRED:", gpt2_translate(text_en))
    print("-"*50)


EN: But suddenly she heard the rustle of a dress and a burst of suppressed sobbing. A pair of arms encircled her neck from below and Kitty was kneeling before her.
GT: Но вдруг она услыхала шум платья и вместе звук разразившегося сдержанного рыданья, и чьи-то руки снизу обняли ее шею. Кити на коленях стояла пред ней.
PRED:  But suddenly she heard the rustle of a dress and a burst of suppressed sobbing. A pair of arms encircled her neck from below and Kitty was kneeling before her.  Она была была была была была была была была была была была была была была. 
--------------------------------------------------
EN: 'Yes, tell me what is happening in Pokrovsk Is the house still standing, and the birch trees, and our schoolroom?
GT: -- Да расскажи мне, что делается в Покровском? Что, дом все стоит, и березы, и наша классная?
PRED:  'Yes, tell me what is happening in Pokrovsk Is the house still standing, and the birch trees, and our schoolroom?  -- Да, что не можение, что не можение, что не мо

In [ ]:
t5_model_lora.eval()
t5_model_lora.to(device)

def t5_translate(text):
    input_ids = t5_tokenizer.encode(
        f"translate English to Russian: {text}",
        return_tensors="pt",
        truncation=True
    ).to(device)

    output_ids = t5_model_lora.base_model.generate(
        input_ids,
        max_length=200,
        num_beams=5
    )
    return t5_tokenizer.decode(output_ids[0], skip_special_tokens=True)

for i in range(5):
    ex = t5_test[i]
    text_en = ex['input_text'].replace("translate English to Russian: ", "")
    text_gt = ex['target_text']
    print("EN:", text_en)
    print("GT:", text_gt)
    print("PRED:", t5_translate(text_en))
    print("-"*50)


EN: But suddenly she heard the rustle of a dress and a burst of suppressed sobbing. A pair of arms encircled her neck from below and Kitty was kneeling before her.
GT: Но вдруг она услыхала шум платья и вместе звук разразившегося сдержанного рыданья, и чьи-то руки снизу обняли ее шею. Кити на коленях стояла пред ней.
PRED: о все она она она лаала и лаала оеала и оеала оеала, которе оеала ее и лексе лександрови, и лександрови лександрови ее.
--------------------------------------------------
EN: 'Yes, tell me what is happening in Pokrovsk Is the house still standing, and the birch trees, and our schoolroom?
GT: -- Да расскажи мне, что делается в Покровском? Что, дом все стоит, и березы, и наша классная?
PRED: -- а, то то, то то то оео в ркади, то оео оео, и оео оео и оео?
--------------------------------------------------
EN: 'Matthew!' he called, 'will you and Mary arrange everything for Anna Arkadyevna in the little sitting-room?' he added when Matthew appeared.
GT: -- Матвей!-- крикн

In [37]:
t5_model_lora.print_trainable_parameters()


trainable params: 294,912 || all params: 60,801,536 || trainable%: 0.4850


In [46]:
import transformers
print(transformers.__version__)


4.57.1


In [ ]:
from transformers import TrainingArguments

training_args_t5 = TrainingArguments(
    output_dir="./t5_lora",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=25,
    learning_rate=1e-4,
    logging_steps=50,
    save_total_limit=1,
    fp16=False,
)



t5_lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)

trainer_t5 = Trainer(
    model=t5_model_lora,
    args=training_args_t5,
    train_dataset=t5_train_tok,
    eval_dataset=t5_test_tok,
    tokenizer=t5_tokenizer,
)

C:\Users\igor\AppData\Local\Temp\ipykernel_20144\4138070477.py:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_t5 = Trainer(


In [55]:
trainer_t5.train()


Step,Training Loss
50,0.316300
100,0.292900
150,0.329000
200,0.313200
250,0.289700
300,0.322500
350,0.301200
400,0.322500
450,0.338800
500,0.320700


TrainOutput(global_step=87475, training_loss=0.30234935961099446, metrics={'train_runtime': 8212.4788, 'train_samples_per_second': 42.606, 'train_steps_per_second': 10.651, 'total_flos': 4.76730951204864e+16, 'train_loss': 0.30234935961099446, 'epoch': 25.0})

In [ ]:
t5_model_lora.eval()
t5_model_lora.to(device)

def t5_translate(text):
    input_ids = t5_tokenizer.encode(
        f"translate English to Russian: {text}",
        return_tensors="pt",
        truncation=True
    ).to(device)

    output_ids = t5_model_lora.base_model.generate(
        input_ids,
        max_length=200,
        num_beams=5
    )
    return t5_tokenizer.decode(output_ids[0], skip_special_tokens=True)

for i in range(5):
    ex = t5_test[i]
    text_en = ex['input_text'].replace("translate English to Russian: ", "")
    text_gt = ex['target_text']
    print("EN:", text_en)
    print("GT:", text_gt)
    print("PRED:", t5_translate(text_en))
    print("-"*50)


EN: But suddenly she heard the rustle of a dress and a burst of suppressed sobbing. A pair of arms encircled her neck from below and Kitty was kneeling before her.
GT: Но вдруг она услыхала шум платья и вместе звук разразившегося сдержанного рыданья, и чьи-то руки снизу обняли ее шею. Кити на коленях стояла пред ней.
PRED: о вседа она слуала а аа и еловек, еловек еловек, и ити еловелас на нее.
--------------------------------------------------
EN: 'Yes, tell me what is happening in Pokrovsk Is the house still standing, and the birch trees, and our schoolroom?
GT: -- Да расскажи мне, что делается в Покровском? Что, дом все стоит, и березы, и наша классная?
PRED: -- а, увствовал, то то в ерео аео, и ереа ереа и ереео?
--------------------------------------------------
EN: 'Matthew!' he called, 'will you and Mary arrange everything for Anna Arkadyevna in the little sitting-room?' he added when Matthew appeared.
GT: -- Матвей!-- крикнул он, -- так устрой же все там с Марьей в диванной для 